In [2]:
!pip install katpoint


[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
#import datetime
import katpoint
import numpy as np
import time as time
import pandas as pd
import math
import colorama
from colorama import Fore
import pytest
from datetime import date 

ModuleNotFoundError: No module named 'katpoint'

We compute the rise and set time of the Sun for the given timestamps. This helps us to define the day and the night time. For the purpose of this script we are only interested in identifying the night time for every timestamps. We plot the LST against the date to show the rise and set of the Sun.

In [46]:

def rise_set(timestamp):
    site = katpoint.Antenna("m000, -30:42:39.8, 21:26:38.0, 1086.6, 13.5") # the site position using one antenna(m000)
    site.observer.date=katpoint.Timestamp(timestamp).to_ephem_date() # Set time
    site.observer.horizon='00:00:00.0' # Set horizon
    observer = site.observer 
    Sun = katpoint.Target("Sun,special")# Make sun object
    ephem_sun = Sun.body #Sun as an object
    ephem_sun.compute(observer) #compute for position of the observer
    return observer.next_rising(ephem_sun),observer.next_setting(ephem_sun)


In [48]:

start_date = "2023-10-01"
end_date = "2023-11-01"
date_start = dt.datetime.strptime(start_date, '%Y-%m-%d')
date_end = dt.datetime.strptime(end_date, '%Y-%m-%d')

d = (date_end.day - date_start.day)
date1 = date(date_start.year, date_start.month, date_start.day)
date2 = date(date_end.year, date_end.month, date_end.day)
delta = date2 - date1
print(date1.day)

1


In [57]:
def sun_set_rise(start_date, end_date):
    
    date_start = dt.datetime.strptime(start_date, '%Y-%m-%d')
    date_end = dt.datetime.strptime(end_date, '%Y-%m-%d')

    date1 = date(date_start.year, date_start.month, date_start.day)
    date2 = date(date_end.year, date_end.month, date_end.day)
    delta = date2 - date1
    days = delta.days
    
    settime, risetime = np.zeros((days,1)),np.zeros((days,1))
    site = katpoint.Antenna("m000, -30:42:39.8, 21:26:38.0, 1086.6, 13.5")
    S = []
    R = []
    D = []
    for day in range(date1.day, date2.day):
        time_ = dt.datetime(date_start.year, date_start.month, day)
        t = dt.datetime.timestamp(time_)
        if day == 1:
            timestamp = t
        else:
            timestamp = t + 24*60*60*day
        rise_time, set_time = rise_set(timestamp)
        risetime[day] = np.degrees(site.local_sidereal_time(rise_time))/15.
        settime[day] = np.degrees(site.local_sidereal_time(set_time))/15.
        S.append(settime[day])
        R.append(risetime[day])
        D.append(time_)
        
    calender = [{"date": D,
        "lst_sunrise":R,
        "lst_sunset": S}
               ]
        
    return calender

In [58]:
start_date = "2024-01-01"
end_date = "2024-12-31"

s = sun_set_rise(start_date, end_date)

IndexError: index 26 is out of bounds for axis 0 with size 5

In [56]:
s

[{'date': [], 'lst_sunrise': [], 'lst_sunset': []}]

## def get_date(start, end):
    start = datetime(2023,1,1)
    end = datetime(2023,2,1)

    calendar = [
    {
        "dt": start + timedelta(days=d),
        "lst_start": "calc this",
        "lst_end": "calc this"
    }
    for d in range(0, (end - start).days + 1)
]

function sun_set_rise(year, month, days)
    
    """
    Calculate sunrise and sunset times for a given location and date range.

    Parameters:
    -------------
    year : int
        The year for which you want to calculate sunrise and sunset times.
    month : int
        The month for which you want to calculate sunrise and sunset times.
    days : int
        The number of days for which you want to calculate sunrise and sunset times.

    Returns:
    -------------
    settime : 2D array of (Timestamp, LST) pairs
        A 2D array of sunset times and their corresponding Local Sidereal Time (LST).
    risetime : 2D array of (Timestamp, LST) pairs
        A 2D array of sunrise times and their corresponding Local Sidereal Time (LST).
    """
    
    # Initialize 2D arrays to store sunset and sunrise times
    settime = create_empty_array(days, 2)
    risetime = create_empty_array(days, 2)
    
    # Define the location of the observation site (e.g., latitude, longitude, altitude)
    site_location = [latitude, longitude, altitude]
    
    # Iterate through a range of days
    for day in range(1, days):
        # Create a datetime object for the current day
        current_date = create_datetime(year, month, day)
        
        # Calculate the timestamp for the current date
        timestamp = calculate_timestamp(current_date, day)
        
        # Calculate the sunrise and sunset times for the current date and location
        rise_time, set_time = calculate_sunrise_sunset(timestamp, site_location)
        
        # Convert the times to Timestamp objects and calculate Local Sidereal Time (LST)
        risetime[day] = (create_timestamp(rise_time), calculate_local_sidereal_time(rise_time, site_location) / 15.0)
        settime[day] = (create_timestamp(set_time), calculate_local_sidereal_time(set_time, site_location) / 15.0)
    
    return settime, risetime


Identifying the night by taking the LST set and rise for each timestamp. 

In [61]:
def get_nighttime(riset,sett):
    'Arranging LST rise and set time to show night time '
    ind1 = 0
    ind2 = -1
       
    rt_new = np.delete(riset, ind1)
    st_new = np.delete(sett,ind2)
    set_rise = {}
    for i in range(len(riset)-1):
        set_rise[i] = [st_new[i],rt_new[i]]
    return set_rise

In [65]:
sr = nighttime(s,r)
sr

{0: [0.0, 0.0],
 1: [0.0, 1698857665.804886],
 2: [1698810097.87872, 21.051453271626],
 3: [7.801962514512852, 1698944113.738086],
 4: [1698896446.914639, 21.1305162676988],
 5: [7.853478883523149, 1699030562.053562],
 6: [1698982797.197651, 21.209685385385118],
 7: [7.9053424356242, 1699117010.74065],
 8: [1699069148.757262, 21.288957422485534],
 9: [7.957561095405107, 1699203459.786671],
 10: [1699155501.622203, 21.36832873142497],
 11: [8.01014268666651, 1699289909.177005],
 12: [1699241855.820418, 21.447795231735725],
 13: [8.063094926191786, 1699376358.895194],
 14: [1699328211.379039, 21.527352426819668],
 15: [8.116425407171844, 1699462808.922969],
 16: [1699414568.324389, 21.606995401232094],
 17: [8.170141587882155, 1699549259.240305],
 18: [1699500926.681931, 21.68671882445191],
 19: [8.224250767547264, 1699635709.825368],
 20: [1699587286.47619, 21.766516919202374],
 21: [8.278760051446529, 1699722160.654493],
 22: [1699673647.730671, 21.846383423819024],
 23: [8.33367630667

function nighttime(riset, sett)
    # Description: Arranging LST rise and set time to show night time

    ind1 = 0
    ind2 = -1

    # Remove the first element from 'riset' and the last element from 'sett'
    rt_new = remove_element(riset, ind1)
    st_new = remove_element(sett, ind2)

    # Create an empty dictionary to store the night time pairs
    setrise = {}

    # Iterate through the elements of 'riset' and 'sett' except the last element
    for i in range(length(riset) - 1):
        setrise[i] = [st_new[i], rt_new[i]

    return set_rise

# Helper function to remove an element from a list
function remove_element(arr, index)
    return arr[:index] + arr[index+1:]


Given csv file constaining observational information for observation schedule blocks, the function extract the night time observations only. 

In [116]:
def nightobs(filename, datefile):
    """Extracting only night time observations\
    from a given file of observational information
    
        Parmeters:
    -------------
    filename : csv file
        file containing all observations
    s_rise : float
        LST sun rise
    """
    
    ndates = []
    for j in range(len(datefile)):
        ndates.append(datefile[j].strftime('%Y-%m-%d'))
    data = pd.read_csv(filename)
    night_obs = data['night_obs']
    LST_starts = data['lst_start']
    LST_ends = data['lst_start_end']
    duration = data['simulated_duration']
    proposal_id = data['proposal_id']
    #proposal_id = sorted(proposal_id)
    ID = data['id']
    
    obs = {}
    for i in range(len(night_obs)):
        if night_obs[i] == 'Yes':
            obs[ID[i]] = [proposal_id[i],LST_starts[i],LST_ends[i],duration[i]]
    return obs, ndates

function nightobs(filename, datefile)
    
    """
    Extracting only night-time observations from a given file of observational information

    Parameters:
    -------------
    filename : csv file
        File containing all observations
    datefile : list of dates
        List of dates to filter observations
    """
    
    # Initialize an empty list to store formatted dates
    ndates = []

    # Convert dates in datefile to a specific format and store them in ndates
    for j in range(length(datefile)):
        ndates.append(datefile[j].strftime('%Y-%m-%d'))

    # Read data from the CSV file
    data = read_csv(filename)

    # Extract relevant columns from the data
    night_obs = data['night_obs']
    LST_starts = data['lst_start']
    LST_ends = data['lst_start_end']
    duration = data['simulated_duration']
    proposal_id = data['proposal_id']
    ID = data['id']
    
    # Initialize an empty dictionary to store filtered observations
    obs = {}

    # Iterate through the night_obs column to filter night-time observations
    for i in range(length(night_obs)):
        if night_obs[i] == 'Yes':
            obs[ID[i]] = [proposal_id[i], LST_starts[i], LST_ends[i], duration[i]]

    return obs, ndates


Pass a csv file in order to run parameter_extract function. This cell gives us LST start, end and duration of the observation in the same format. 

In [117]:
def format_params(obs):
    
    duration_ = []
    LST_start = []
    LST_end = []
    obs_id = []
    for i in obs.values():
        duration_.append(round(i[3]/3600,2))    
        g1 = i[1].replace(':', '.')
        f1 = float(g1)
        LST_start.append(f1)
        g2 = i[2].replace(':', '.') 
        f2 = float(g2)
        LST_end.append(f2)
        obs_id.append(i[0])

    duration_ = [0 if math.isnan(x) else x for x in duration_]

    return LST_start, LST_end, duration_, obs_id

function format_params(obs)
    
    """
    Format observation parameters.

    Parameters:
    -------------
    obs : dictionary
        Dictionary containing observation data

    Returns:
    -------------
    LST_start : list
        List of formatted LST start times
    LST_end : list
        List of formatted LST end times
    duration_ : list
        List of formatted durations
    obs_id : list
        List of observation IDs
    """

    # Initialize empty lists to store formatted data
    duration_ = []
    LST_start = []
    LST_end = []
    obs_id = []

    # Iterate through the values in the obs dictionary
    for i in obs.values():
        # Format and append duration
        duration_.append(round(i[3] / 3600, 2))

        # Format and append LST start time
        g1 = i[1].replace(':', '.')
        f1 = float(g1)
        LST_start.append(f1)

        # Format and append LST end time
        g2 = i[2].replace(':', '.')
        f2 = float(g2)
        LST_end.append(f2)

        # Append observation ID
        obs_id.append(i[0])

    # Replace any NaN values in duration_ with 0
    duration_ = [0 if math.isnan(x) else x for x in duration_]

    return LST_start, LST_end, duration_, obs_id


We check if the observation time range is within the LST night time range and color each observation by red if is not(cant run at that day due to both its LST start and end being outside of the night time zone), yellow if it is not(cant run at that day due to its LST end being outside of the range) and green if it is(can run at that day).

In [118]:
def obs_selection(s_set, s_rise, start_r, end_r, dur, observation_name):
    """ Check if the observation time range is within the LST time range.
    
    Parmeters:
    -------------
    s_set : float
        LST sun set
    s_rise : float
        LST sun rise
    start_r : float
        observation allowed start LST range
    end_r : float
        observation allowed end LST range
    dur : float 
        observation duration in hours
    """
    # check if observation start or end time is within the LST range
    cond1 = start_r >= s_set
    cond2 = end_r > s_set
    if  cond1 or cond2:
        if (cond1 and cond2) or cond1:
            start_time = start_r
            end_time = start_time + dur
        else:
            start_time = end_r
            end_time = start_time + dur
        if end_time > s_rise and end_time >= 24:
            return (Fore.RED + '{} observation end time ({}) is outside the LST range'.format(observation_name, end_time-dur))
        else:
            if end_time >= 24:
                end_time = end_time - 24
    else:
        return (Fore.YELLOW + '{} both start({}) and the end time ({}) are outside the LST range'.format(observation_name, start_time, end_time-dur))
    
    
    return (Fore.GREEN + 'Start time for {} is {} and end_time is {}'.format(observation_name, start_time, end_time))
    

function obs_selection(s_set, s_rise, start_r, end_r, dur, observation_name)
    
    """
    Select observations based on LST ranges.

    Parameters:
    -------------
    s_set : float
        LST time of sunset
    s_rise : float
        LST time of sunrise
    start_r : float
        Observation start time
    end_r : float
        Observation end time
    dur : float
        Duration of the observation
    observation_name : str
        Name of the observation

    Returns:
    -------------
    str
        A message indicating the status of the observation selection
    """

    # Check if observation start or end time is within the LST range
    condition1 = start_r >= s_set
    condition2 = end_r > s_set

    if condition1 or condition2:
        if condition1 and condition2 or condition1:
            start_time = start_r
            end_time = start_time + duration
        else:
            start_time = end_r
            end_time = start_time + duration

        if end_time > s_rise and end_time >= 24:
            return (Fore.RED + '{} observation end time ({}) is outside the LST range'.format(observation_name, end_time - duration))
        else:
            if end_time >= 24:
                end_time = end_time - 24
    else:
        return (Fore.YELLOW + '{} both start({}) and the end time ({}) are outside the LST range'.format(observation_name, start_time, end_time - duration))

    return (Fore.GREEN + 'Start time for {} is {} and end_time is {}'.format(observation_name, start_time, end_time))


In [122]:
from astral.sun import sun
from astral import LocationInfo

def sun_stats(dt, lat, lng):
    location = LocationInfo(latitude=latitude, longitude=longitude)
    location.timezone = "UTC" # I find fixing dates to utc reassuring and helpful
    return sun(location.observer, date=dt)

ModuleNotFoundError: No module named 'astral'

In [3]:
from datetime import datetime, timedelta

start = datetime(2023,1,1)
end = datetime(2023,2,1)

calendar = [
    {
        "dt": start + timedelta(days=d),
        "lst_start": "calc this",
        "lst_end": "calc this"
    }
    for d in range(0, (end - start).days + 1)
]

In [4]:
calendar

[{'dt': datetime.datetime(2023, 1, 1, 0, 0),
  'lst_start': 'calc this',
  'lst_end': 'calc this'},
 {'dt': datetime.datetime(2023, 1, 2, 0, 0),
  'lst_start': 'calc this',
  'lst_end': 'calc this'},
 {'dt': datetime.datetime(2023, 1, 3, 0, 0),
  'lst_start': 'calc this',
  'lst_end': 'calc this'},
 {'dt': datetime.datetime(2023, 1, 4, 0, 0),
  'lst_start': 'calc this',
  'lst_end': 'calc this'},
 {'dt': datetime.datetime(2023, 1, 5, 0, 0),
  'lst_start': 'calc this',
  'lst_end': 'calc this'},
 {'dt': datetime.datetime(2023, 1, 6, 0, 0),
  'lst_start': 'calc this',
  'lst_end': 'calc this'},
 {'dt': datetime.datetime(2023, 1, 7, 0, 0),
  'lst_start': 'calc this',
  'lst_end': 'calc this'},
 {'dt': datetime.datetime(2023, 1, 8, 0, 0),
  'lst_start': 'calc this',
  'lst_end': 'calc this'},
 {'dt': datetime.datetime(2023, 1, 9, 0, 0),
  'lst_start': 'calc this',
  'lst_end': 'calc this'},
 {'dt': datetime.datetime(2023, 1, 10, 0, 0),
  'lst_start': 'calc this',
  'lst_end': 'calc this'},

In [3]:

from typing import Union
from astral.sun import sun
from astral import LocationInfo
from normalize_coordinates import normalize_coordinates
from .normalize_date import normalize_yyyymmdd_to_datetime


def sun_stats(latitude: Union[float, str], longitude: Union[float, str], yyyymmdd: str) -> dict:

    latitude, longitude = normalize_coordinates(latitude, longitude)
    dt = normalize_yyyymmdd_to_datetime(yyyymmdd)
    location = LocationInfo(latitude=latitude, longitude=longitude)
    location.timezone = "UTC"
    return sun(location.observer, date=dt)

ModuleNotFoundError: No module named 'normalize_coordinates'

In [4]:
from astral.sun import sun
from astral import LocationInfo

def sun_stats(dt, lat, lng):
    location = LocationInfo(latitude=latitude, longitude=longitude)
    location.timezone = "UTC" # I find fixing dates to utc reassuring and helpful
    return sun(location.observer, date=dt)

In [5]:

s = sun_stats(datetime(2023,1,1),-30:42:39.8, 21:26:38.0)

SyntaxError: invalid syntax (<ipython-input-5-9b329407a358>, line 1)

In [7]:
location = LocationInfo(latitude=30.7130, longitude=21.4430)

In [8]:
location

LocationInfo(name='Greenwich', region='England', timezone='Europe/London', latitude=30.713, longitude=21.443)

In [9]:
sun(location.observer, date=dt)

AttributeError: module 'datetime' has no attribute 'year'